# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from barcode_runs_Crowe_antibodies.csv
Here are the first few entries in the Illumina runs:


library                       sample    date  experiment antibody  \
0    lib1  expt_13-16-none-0-reference  200720  expt_13-16     none   
1    lib2  expt_13-16-none-0-reference  200720  expt_13-16     none   
2    lib1   expt_7-11-none-0-reference  200616   expt_7-11     none   
3    lib2   expt_7-11-none-0-reference  200616   expt_7-11     none   
4    lib1  expt_22-23-none-0-reference  200810  expt_22-23     none   

   concentration concentration_units group  selection  frac_escape  \
0              0           ng_per_mL  none  reference          NaN   
1              0           ng_per_mL  none  reference          NaN   
2              0           ng_per_mL  none  reference          NaN   
3              0           ng_per_mL  none  reference          NaN   
4              0           ng_per_mL  none  reference          NaN   

   cells_sorted  \
0   160000000.0   
1   160000000.0   
2   160000000.0   
3   160000000.0   
4   160000000.0   

                                                                                                                                R1  
0  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/*R1*.fastq.gz  
1  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib2_ref/*R1*.fastq.gz  
2  /shared/ngs/illumina/agreaney/200618_D00300_0980_AHFCMJBCX3/Unaligned/Project_agreaney/Sample_expt_7-11_lib1_ACE2pos/*.fastq.gz  
3  /shared/ngs/illumina/agreaney/200618_D00300_0980_AHFCMJBCX3/Unaligned/Project_agreaney/Sample_expt_7-11_lib2_ACE2pos/*.fastq.gz  
4  /shared/ngs/illumina/agreaney/200812_D00300_1026_BHFFJTBCX3/Unaligned/Project_agreaney/Sample_expt_22-23_lib1_ref/*R1*.fastq.gz

Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


biosample_accession                        library_ID  \
0        SAMN16054076  lib1_expt_13-16-none-0-reference   
1        SAMN16054076  lib1_expt_13-16-none-0-reference   
2        SAMN16054076  lib1_expt_13-16-none-0-reference   
3        SAMN16054076  lib1_expt_13-16-none-0-reference   
4        SAMN16054076  lib1_expt_13-16-none-0-reference   

                                                                                         title  \
0  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
1  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
2  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
3  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
4  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   

  library_strategy library_source library_selection library_layout  platform  \
0         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
1         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
2         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
3         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
4         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   

      instrument_model                 design_description filetype  \
0  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
1  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
2  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
3  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
4  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   

                                                                                                                                                     filename_fullpath  \
0  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/expt_13-16_lib1_ref_AAGGTACA_L001_R1_002.fastq.gz   
1  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/expt_13-16_lib1_ref_AAGGTACA_L002_R1_004.fastq.gz   
2  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/expt_13-16_lib1_ref_AAGGTACA_L002_R1_002.fastq.gz   
3  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/expt_13-16_lib1_ref_AAGGTACA_L001_R1_005.fastq.gz   
4  /shared/ngs/illumina/agreaney/200727_D00300_1012_AHFFMGBCX3/Unaligned/Project_agreaney/Sample_expt_13-16_lib1_ref/expt_13-16_lib1_ref_AAGGTACA_L002_R1_005.fastq.gz   

                                            filename  
0  expt_13-16_lib1_ref_AAGGTACA_L001_R1_002.fastq.gz  
1  expt_13-16_lib1_ref_AAGGTACA_L002_R1_004.fastq.gz  
2  expt_13-16_lib1_ref_AAGGTACA_L002_R1_002.fastq.gz  
3  expt_13-16_lib1_ref_AAGGTACA_L001_R1_005.fastq.gz  
4  expt_13-16_lib1_ref_AAGGTACA_L002_R1_005.fastq.gz

For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

biosample_accession                         library_ID  n_files
0         SAMN16054076   lib1_expt_22-23-none-0-reference       34
1         SAMN16054076   lib2_expt_22-23-none-0-reference       32
2         SAMN16054076     lib1_expt_3-6-none-0-reference       16
3         SAMN16054076     lib2_expt_3-6-none-0-reference       16
4         SAMN16054076    lib2_expt_7-11-none-0-reference       16
5         SAMN16054076    lib1_expt_7-11-none-0-reference       16
6         SAMN16054076   lib1_expt_13-16-none-0-reference       12
7         SAMN16054076   lib2_expt_13-16-none-0-reference       10
8         SAMN16054076  lib2_expt_15-COV2-2499-400-escape        4
9         SAMN16054076  lib2_expt_23-COV2-2096-400-escape        4
10        SAMN16054076  lib2_expt_13-COV2-2050-400-escape        4
11        SAMN16054076  lib1_expt_23-COV2-2096-400-escape        4
12        SAMN16054076  lib1_expt_15-COV2-2499-400-escape        4
13        SAMN16054076   lib2_expt_5-COV2-2479-400-escape        3
14        SAMN16054076  lib2_expt_14-COV2-2094-400-escape        2
15        SAMN16054076      lib2_expt_3-CR3022-400-escape        2
16        SAMN16054076  lib2_expt_16-COV2-2677-400-escape        2
17        SAMN16054076  lib1_expt_10-COV2-2832-400-escape        2
18        SAMN16054076  lib2_expt_10-COV2-2832-400-escape        2
19        SAMN16054076  lib2_expt_11-COV2-2082-400-escape        2
20        SAMN16054076  lib1_expt_11-COV2-2082-400-escape        2
21        SAMN16054076   lib1_expt_9-COV2-2165-400-escape        2
22        SAMN16054076   lib1_expt_5-COV2-2479-400-escape        2
23        SAMN16054076      lib1_expt_3-CR3022-400-escape        2
24        SAMN16054076  lib1_expt_16-COV2-2677-400-escape        2
25        SAMN16054076  lib1_expt_14-COV2-2094-400-escape        2
26        SAMN16054076  lib1_expt_13-COV2-2050-400-escape        2
27        SAMN16054076   lib2_expt_9-COV2-2165-400-escape        2

Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


biosample_accession                        library_ID  \
0        SAMN16054076  lib1_expt_13-16-none-0-reference   
1        SAMN16054076  lib2_expt_13-16-none-0-reference   
2        SAMN16054076   lib1_expt_7-11-none-0-reference   
3        SAMN16054076   lib2_expt_7-11-none-0-reference   
4        SAMN16054076  lib1_expt_22-23-none-0-reference   

                                                                                         title  \
0  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
1  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_13-16-none-0-reference   
2   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_7-11-none-0-reference   
3   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_7-11-none-0-reference   
4  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_22-23-none-0-reference   

  library_strategy library_source library_selection library_layout  platform  \
0         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
1         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
2         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
3         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
4         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   

      instrument_model                 design_description  ...  \
0  Illumina HiSeq 2500  PCR of barcodes from RBD variants  ...   
1  Illumina HiSeq 2500  PCR of barcodes from RBD variants  ...   
2  Illumina HiSeq 2500  PCR of barcodes from RBD variants  ...   
3  Illumina HiSeq 2500  PCR of barcodes from RBD variants  ...   
4  Illumina HiSeq 2500  PCR of barcodes from RBD variants  ...   

                                          filename25  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L001_R1_014.fastq.gz   

                                          filename26  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L002_R1_009.fastq.gz   

                                          filename27  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L001_R1_001.fastq.gz   

                                          filename28  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L002_R1_001.fastq.gz   

                                          filename29  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L002_R1_011.fastq.gz   

                                          filename30  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  expt_22-23_lib1_ref_ACCACTGT_L001_R1_004.fastq.gz   

                                          filename31  \
0                                                NaN   
1                                             

Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
